In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_train=pd.read_csv("../input/train_5UKooLv.csv")
df_test=pd.read_csv('../input/test_J1hm2KQ.csv')

In [3]:
test_customers=pd.DataFrame(df_test.CustomerID.unique(),columns=["CustomerID"])

In [4]:
df_train=df_train.append(df_test,sort=False)

In [5]:
df_train.Quantity=df_train.Quantity.abs()
df_train.UnitPrice=df_train.UnitPrice.abs()

In [6]:
unique_stock=df_train.StockCode.unique()

In [7]:
len(unique_stock)

3860

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433672 entries, 0 to 103096
Data columns (total 7 columns):
CustomerID     433672 non-null int64
InvoiceNo      433672 non-null int64
Quantity       433672 non-null int64
InvoiceDate    433672 non-null object
UnitPrice      433672 non-null float64
Country        433672 non-null object
StockCode      433672 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 26.5+ MB


In [9]:
df_train["expenditure"]=df_train.Quantity*df_train.UnitPrice
df_train.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode,expenditure
0,27270,27270,7,01/12/10 8:26,2.55,PX,85123AY,17.85
1,27270,27270,7,01/12/10 8:26,3.39,PX,71053R,23.73
2,27270,27270,9,01/12/10 8:26,2.75,PX,84406BH,24.75
3,27270,27270,7,01/12/10 8:26,3.39,PX,84029GV,23.73
4,27270,27270,7,01/12/10 8:26,3.39,PX,84029EX,23.73


In [10]:
t=df_train.InvoiceDate.str.split('/', expand = True)
df_train.InvoiceDate=t[1]
df_train.InvoiceDate=df_train.InvoiceDate.astype('int')
df_train.InvoiceDate=df_train.InvoiceDate.astype('str')
df_train.InvoiceDate.value_counts()

11    67530
12    54654
10    49835
9     39978
7     30912
6     29849
3     29064
5     28982
1     28663
8     28256
4     23911
2     22038
Name: InvoiceDate, dtype: int64

### Procedure:

- We are first going to find the similarity between users 
- Then, based on the frequency at which  a user purchases a particular product we will construct a user-item matrix having a users weighted preference score for every unique product in the list
- For a given user id we will first extract 10 most similar users
- Following this we will create a list of recommended product based on the user-item matrix we had obtained

#### `Factors considered while estimating user similarity`
- frequency at which he purchases a certain product during a certain time of the year
- frequency of making a purchase while he is in a certain country during a certain time of the year 

In [11]:
df_new=df_train

In [12]:
df_new.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode,expenditure
0,27270,27270,7,12,2.55,PX,85123AY,17.85
1,27270,27270,7,12,3.39,PX,71053R,23.73
2,27270,27270,9,12,2.75,PX,84406BH,24.75
3,27270,27270,7,12,3.39,PX,84029GV,23.73
4,27270,27270,7,12,3.39,PX,84029EX,23.73


### svd recommendation

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [14]:
df_ui=pd.pivot_table(df_train,index="CustomerID",columns="StockCode",values="Quantity")
df_ui.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90205AR,90205CD,90206AQ,90206CD,90208D,90209AX,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
mean_user_purchases=np.array(df_ui.mean(axis=1))
mean_user_purchases

array([ 6.53494624,  9.09939153, 15.24903846, ...,  5.33333333,
       18.        , 10.44444444])

In [16]:
df_ui.fillna(0,inplace=True)

In [17]:
from scipy.sparse.linalg import svds 

In [18]:
U, sigma, Vt = svds(df_ui, k = 20)

In [19]:
sigma=np.flip(sigma)
sigma = np.diag(sigma)

In [20]:
all_user_predictions= np.dot(np.dot(U, sigma), Vt) +mean_user_purchases.reshape(-1, 1)
preds = pd.DataFrame(all_user_predictions, columns = df_ui.columns,index=df_ui.index)
preds.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90205AR,90205CD,90206AQ,90206CD,90208D,90209AX,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10.165455,9.390871,7.779671,6.581970,6.664893,6.608294,5.993481,20.439101,16.996430,-21.958578,7.191882,227.707274,-582.898503,65.633131,12.056360,11.815707,11.110264,13.503211,7.578391,-2.730134,10.170916,6.603009,6.744419,6.522767,8.722085,6.649323,13.954788,-1.730439,6.486650,120.082635,6.635961,10.895222,8.328928,-5.780977,12.636948,9.716348,6.258021,10.232892,23.646481,207.557914,...,7.012653,6.976665,6.683011,6.651005,7.446987,7.446691,7.488836,7.957443,5.628122,4.927587,6.999322,9.307144,6.622229,6.530496,6.591162,6.571904,6.830567,6.541558,6.605822,6.479831,6.542064,6.393113,6.492929,6.517454,6.543755,6.794633,6.557568,6.424504,6.626146,6.399486,6.365514,6.314835,6.744832,6.702783,6.394208,6.357200,6.469102,6.381219,6.554292,6.402748
900,12.196322,11.365098,10.022730,9.139209,9.199307,9.165235,9.070015,20.244224,17.654112,-9.065191,9.642363,132.344208,-455.838717,42.963382,13.591506,13.277246,12.768891,14.541345,9.973913,2.166144,12.103419,9.166498,9.300849,9.097654,10.925688,9.208352,13.384033,3.229042,9.064119,95.246702,9.192169,13.197326,10.538567,-11.992145,13.159414,11.854351,8.916120,11.991814,18.746047,132.220657,...,9.468191,9.428128,9.214387,9.196774,9.801404,9.786818,9.858298,10.160712,8.521235,7.949046,9.511832,11.178224,9.171928,9.100658,9.145293,9.129493,9.368155,9.139157,9.196794,9.088487,9.129977,9.018620,9.097809,9.109697,9.142662,9.322915,9.149865,9.093235,9.196074,9.017507,8.993868,8.963295,9.282575,9.257457,9.018350,8.987308,9.073825,9.006614,9.175676,9.025381
1800,31.191551,28.148812,20.241264,15.454840,15.826115,15.589226,12.019101,76.719242,64.142025,-112.571812,18.566351,671.278804,-2868.979679,200.456971,45.170161,43.733139,40.183257,49.751676,16.438395,-36.056001,31.068673,15.535045,17.015215,15.273519,25.810685,16.080759,39.679670,-26.919364,14.984131,502.762850,15.699479,42.279868,20.715621,-55.464167,49.077642,40.558197,13.542110,30.919440,67.995925,710.286708,...,17.381559,17.234430,15.901729,15.713417,19.478076,19.131005,19.692412,20.731718,11.227770,7.515102,16.509000,27.384219,15.686652,15.261039,15.510164,15.428501,16.707132,15.471972,15.657792,15.108578,15.390465,14.798907,15.223726,15.300441,15.406381,16.478575,15.574556,15.088950,15.776148,14.762097,14.671354,14.398293,16.304682,16.091270,14.812211,14.634329,15.123783,14.744071,16.118027,14.885600
1890,7.299102,7.289211,7.266263,7.250649,7.251768,7.251029,7.240321,7.437971,7.394439,6.831562,7.259284,9.752716,-1.338851,7.935727,7.330865,7.326904,7.316951,7.348276,7.264078,7.111224,7.300033,7.250936,7.253864,7.249925,7.280922,7.252212,7.336821,7.135391,7.249298,8.752684,7.251435,7.316490,7.273194,7.040296,7.335626,7.300746,7.245863,7.300105,7.446871,9.693813,...,7.256519,7.256132,7.252017,7.251527,7.262564,7.262226,7.263384,7.268724,7.236971,7.226779,7.256053,7.287182,7.251279,7.249994,7.250781,7.250519,7.254401,7.250550,7.251279,7.249596,7.250406,7.248472,7.249824,7.250097,7.250468,7.253768,7.250747,7.249242,7.251558,7.248449,7.248083,7.247413,7.253171,7.252595,7.248492,7.247970,7.249484,7.248300,7.251329,7.248639
2700,8.341036,7.961365,7.009743,6.350665,6.398269,6.366939,6.000221,14.275681,12.412276,-10.560990,6.695561,122.442363,-330.343033,37.872375,9.526196,9.385731,8.981883,10.290859,7.318924,1.172385,8.464189,6.365

In [21]:
table=preds.T
index1=preds.columns
col1=preds.index
table = scaler.fit_transform(table)
table=pd.DataFrame(table,index=index1,columns=col1)
table.head()

CustomerID,0,900,1800,1890,2700,2790,3600,3690,4500,4590,5400,5490,6300,6390,7200,7290,8100,8190,9090,9990,10890,11790,11880,12690,12780,13590,13680,14490,14580,15390,15480,16290,16380,17190,17280,18090,18180,19080,19179,19980,...,557145,557235,558045,558135,559035,559935,560835,561735,561825,562635,563535,563625,564435,564525,565335,565425,566235,566325,567135,567225,568035,568125,569025,569925,570825,571725,572625,573525,573615,574425,574515,575325,575415,576225,576315,577125,577215,578025,579015,580815
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002F,0.531304,0.545558,0.560810,0.554666,0.536316,0.539871,0.536504,0.553748,0.354806,0.546234,0.531226,0.550347,0.439685,0.602482,0.538457,0.181038,0.546718,0.566429,0.500170,0.535659,0.536386,0.543261,0.358292,0.296586,0.557568,0.545067,0.572001,0.508376,0.425264,0.541546,0.546919,0.546874,0.364045,0.558805,0.334607,0.578034,0.541534,0.588994,0.508181,0.411601,...,0.441218,0.546698,0.403103,0.528641,0.332616,0.399207,0.165685,0.355329,0.547593,0.545726,0.459764,0.512241,0.517243,0.499917,0.540853,0.389038,0.480032,0.536699,0.551065,0.539138,0.427796,0.536971,0.272490,0.542893,0.539126,0.458989,0.565975,0.537719,0.538128,0.464504,0.440546,0.528081,0.547745,0.554786,0.525115,0.511952,0.394158,0.280505,0.523655,0.549440
10080Q,0.530610,0.544589,0.560222,0.554031,0.535715,0.539052,0.535072,0.552920,0.354533,0.545791,0.530301,0.549579,0.439978,0.601320,0.537621,0.179588,0.546107,0.565678,0.498573,0.535058,0.535463,0.542802,0.357625,0.297660,0.557052,0.544028,0.571801,0.509004,0.425165,0.540745,0.546073,0.545888,0.361839,0.558411,0.334499,0.577123,0.540696,0.588139,0.508934,0.412177,...,0.439731,0.545855,0.403411,0.528083,0.334227,0.399426,0.165622,0.355292,0.547041,0.544966,0.459227,0.511126,0.516584,0.498444,0.540147,0.386485,0.479890,0.535986,0.550467,0.538539,0.427948,0.535773,0.271843,0.541532,0.538524,0.458156,0.565307,0.537024,0.537526,0.464892,0.436091,0.526893,0.547074,0.553925,0.524164,0.511358,0.389788,0.278128,0.522583,0.548774
10120M,0.529167,0.543024,0.558693,0.552558,0.534208,0.537568,0.533561,0.551383,0.355875,0.544268,0.528877,0.548039,0.441328,0.599757,0.536111,0.180113,0.544612,0.564149,0.497223,0.533607,0.533876,0.541237,0.358902,0.298483,0.555521,0.542509,0.570180,0.507662,0.425147,0.539265,0.544567,0.544300,0.361125,0.556885,0.335722,0.575616,0.539219,0.586575,0.510261,0.413548,...,0.438548,0.544264,0.404839,0.526631,0.335060,0.400857,0.165030,0.356633,0.545536,0.543492,0.458096,0.509562,0.515130,0.497020,0.538659,0.386767,0.478719,0.534535,0.548944,0.537086,0.429394,0.534343,0.273029,0.539987,0.537073,0.458725,0.563665,0.535481,0.536030,0.465400,0.435358,0.525392,0.545576,0.552414,0.522705,0.509921,0.388834,0.278174,0.521157,0.547326
10123CU,0.528094,0.541994,0.557767,0.551555,0.533164,0.536547,0.532607,0.550414,0.356536,0.543261,0.527739,0.547063,0.442253,0.598943,0.535055,0.179725,0.543605,0.563185,0.495834,0.532571,0.532789,0.540198,0.359551,0.299099,0.554566,0.541405,0.569491,0.506928,0.425846,0.538256,0.543521,0.543213,0.359489,0.555919,0.336241,0.574626,0.538157,0.585666,0.511379,0.414418,...,0.438085,0.543202,0.405652,0.525557,0.335850,0.401639,0.165178,0.357267,0.544537,0.542460,0.456895,0.508383,0.513915,0.495782,0.537639,0.387109,0.477726,0.533460,0.547971,0.536065,0.430263,0.533238,0.273390,0.538915,0.536055,0.459374,0.562776,0.534510,0.535010,0.466395,0.433596,0.524366,0.544554,0.551280,0.521623,0.508747,0.386884,0.277327,0.520070,0.546279
10124AW,0.528168,0.542064,0.557839,0.551627,0.533240,0.536618,0.532673,0.550492,0.356474,0.543335,0.527813,0.547141,0.442182,0.599000,0.535129,0.179706,0.543677,0.563252,0.495911,0.532645,0.532860,0.540274,0.359484,0.299066,0.554637,0.541475,0.569549,0.506996,0.425791,0.538328,0.543591,0.543282,0.359560,0.555989,0.336188,0.574690,0.538229,0.585728,0.511300,0.414355,...,0.438123,0.543276,0.405587,0.525633,0.335810,0.401574,0.165191,0.357208,0.544609,0.542531,0.456993,0.

In [22]:
preds=table.T

In [23]:
preds.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90205AR,90205CD,90206AQ,90206CD,90208D,90209AX,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.531304,0.530610,0.529167,0.528094,0.528168,0.528118,0.527567,0.540508,0.537424,0.502526,0.528640,0.726192,0.0,0.580996,0.532998,0.532783,0.532151,0.534295,0.528987,0.519752,0.531309,0.528113,0.528240,0.528041,0.530011,0.528154,0.534699,0.520647,0.528009,0.629775,0.528142,0.531958,0.529659,0.517019,0.533518,0.530902,0.527804,0.531365,0.543382,0.708141,...,0.528480,0.528448,0.528185,0.528156,0.528869,0.528869,0.528906,0.529326,0.527240,0.526612,0.528468,0.530535,0.528130,0.528048,0.528102,0.528085,0.528317,0.528058,0.528115,0.528003,0.528058,0.527925,0.528014,0.528036,0.528060,0.528285,0.528072,0.527953,0.528134,0.527931,0.527900,0.527855,0.528240,0.528202,0.527926,0.527893,0.527993,0.527914,0.528069,0.527933
900,0.545558,0.544589,0.543024,0.541994,0.542064,0.542025,0.541914,0.554939,0.551919,0.520775,0.542581,0.685606,0.0,0.581421,0.547184,0.546818,0.546225,0.548291,0.542967,0.533866,0.545449,0.542026,0.542183,0.541946,0.544077,0.542075,0.546942,0.535105,0.541907,0.642364,0.542056,0.546724,0.543625,0.517363,0.546680,0.545159,0.541734,0.545319,0.553192,0.685462,...,0.542378,0.542331,0.542082,0.542061,0.542766,0.542749,0.542832,0.543185,0.541274,0.540607,0.542429,0.544371,0.542032,0.541949,0.542001,0.541983,0.542261,0.541994,0.542061,0.541935,0.541983,0.541854,0.541946,0.541960,0.541998,0.542208,0.542007,0.541941,0.542060,0.541852,0.541825,0.541789,0.542161,0.542132,0.541853,0.541817,0.541918,0.541840,0.542037,0.541862
1800,0.560810,0.560222,0.558693,0.557767,0.557839,0.557793,0.557103,0.569614,0.567182,0.533011,0.558369,0.684585,0.0,0.593541,0.563513,0.563235,0.562549,0.564399,0.557957,0.547807,0.560787,0.557783,0.558069,0.557732,0.559770,0.557888,0.562452,0.549573,0.557676,0.651999,0.557815,0.562954,0.558785,0.544054,0.564269,0.562622,0.557397,0.560758,0.567927,0.692128,...,0.558140,0.558111,0.557854,0.557817,0.558545,0.558478,0.558587,0.558788,0.556950,0.556232,0.557971,0.560074,0.557812,0.557730,0.557778,0.557762,0.558009,0.557771,0.557807,0.557700,0.557755,0.557640,0.557723,0.557737,0.557758,0.557965,0.557790,0.557697,0.557829,0.557633,0.557616,0.557563,0.557932,0.557890,0.557643,0.557609,0.557703,0.557630,0.557896,0.557657
1890,0.554666,0.554031,0.552558,0.551555,0.551627,0.551579,0.550892,0.563584,0.560788,0.524644,0.552110,0.712220,0.0,0.595546,0.556706,0.556452,0.555813,0.557824,0.552417,0.542602,0.554726,0.551574,0.551762,0.551509,0.553499,0.551655,0.557088,0.544154,0.551468,0.648005,0.551606,0.555783,0.553003,0.538048,0.557012,0.554772,0.551248,0.554731,0.564155,0.708437,...,0.551932,0.551907,0.551643,0.551611,0.552320,0.552299,0.552373,0.552716,0.550677,0.550022,0.551902,0.553901,0.551596,0.551513,0.551564,0.551547,0.551796,0.551549,0.551596,0.551487,0.551540,0.551415,0.551502,0.551520,0.551543,0.551755,0.551561,0.551465,0.551614,0.551414,0.551390,0.551347,0.551717,0.551680,0.551417,0.551383,0.551480,0.551404,0.551599,0.551426
2700,0.536316,0.535715,0.534208,0.533164,0.533240,0.533190,0.532609,0.545714,0.542763,0.506384,0.533710,0.716999,0.0,0.583080,0.538193,0.537970,0.537331,0.539404,0.534697,0.524964,0.536511,0.533187,0.533356,0.533116,0.535102,0.533238,0.539320,0.525863,0.533073,0.633019,0.533215,0.537600,0.534771,0.516457,0.538494,0.536087,0.532813,0.536460,0.547354,0.706140,...,0.533548,0.533538,

In [24]:
list(preds.loc[900])

[0.5455576860130648,
 0.5445887833045349,
 0.5430240721900536,
 0.541994210242235,
 0.5420642631618409,
 0.5420245473306506,
 0.5419135559398759,
 0.5549385968748879,
 0.5519194732079985,
 0.520774537644585,
 0.5425807041237776,
 0.6856061802687388,
 0.0,
 0.5814208258274958,
 0.5471839597416314,
 0.5468176478865139,
 0.546225092118847,
 0.5482911253764617,
 0.5429671694653289,
 0.5338661669401227,
 0.5454493951500381,
 0.5420260193139324,
 0.5421826238887796,
 0.5419457728012115,
 0.5440765914662539,
 0.5420748061053228,
 0.5469421225955232,
 0.5351051167387088,
 0.541906682628353,
 0.6423640561048106,
 0.542055942808426,
 0.5467244907977238,
 0.543625349624427,
 0.5173627800158502,
 0.5466802987989924,
 0.5451590725334071,
 0.5417341696758663,
 0.5453193048362446,
 0.5531922701878594,
 0.6854621656318755,
 0.5420543285679893,
 0.5519950198283146,
 0.5287973966263485,
 0.5918482853486918,
 0.5491630114544607,
 0.5429548753035204,
 0.5436116658668878,
 0.5625940730332233,
 0.5351442787

In [25]:
df_new["Custom_col1"]=df_new.Country+df_new.InvoiceDate+" "+df_new.StockCode+df_new.InvoiceDate+" "
df_new.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode,expenditure,Custom_col1
0,27270,27270,7,12,2.55,PX,85123AY,17.85,PX12 85123AY12
1,27270,27270,7,12,3.39,PX,71053R,23.73,PX12 71053R12
2,27270,27270,9,12,2.75,PX,84406BH,24.75,PX12 84406BH12
3,27270,27270,7,12,3.39,PX,84029GV,23.73,PX12 84029GV12
4,27270,27270,7,12,3.39,PX,84029EX,23.73,PX12 84029EX12


In [26]:
grp_fact = {'expenditure': 'sum','Custom_col1': 'sum'}
prod_factor = df_new.groupby('CustomerID', as_index=False).aggregate(grp_fact).reindex(columns=["CustomerID","expenditure","Custom_col1"])

In [27]:
prod_factor.head()

,CustomerID,expenditure,Custom_col1
0,0,5968.56,PX12 21429E12 PX12 21428B12 PX12 22961C12 PX12...
1,900,14257.38,PX12 22943F12 PX12 22118J12 PX12 22121N12 PX12...
2,1800,16962.90,PX12 22655R12 PX12 84985AB12 PX12 22113T12 PX1...
3,1890,100.60,PX12 21094A12 PX12 21080R12 PX4 48194E4 PX4 21...
4,2700,8190.91,PX12 22697A12 PX12 21877C12 PX12 21071P12 PX12...


In [28]:
custid=prod_factor.CustomerID.unique()
custindx=pd.Series(prod_factor.index, index=prod_factor['CustomerID'])
custindx.shape

(1600,)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
vectorizer = CountVectorizer()

In [30]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(2,2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(prod_factor.Custom_col1)

In [31]:
prod_factor.set_index(["CustomerID"],inplace=True)

In [32]:
prod_factor['expenditure'] = pd.qcut(prod_factor['expenditure'], 3,labels=[1,2,3])
prod_factor.expenditure.value_counts()

1    534
3    533
2    533
Name: expenditure, dtype: int64

In [33]:
x=pd.DataFrame(tfidf_matrix.todense(),index=custid)

In [34]:
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,124196,124197,124198,124199,124200,124201,124202,124203,124204,124205,124206,124207,124208,124209,124210,124211,124212,124213,124214,124215,124216,124217,124218,124219,124220,124221,124222,124223,124224,124225,124226,124227,124228,124229,124230,124231,124232,124233,124234,124235
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
x["expenditure"]=prod_factor.expenditure
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,124197,124198,124199,124200,124201,124202,124203,124204,124205,124206,124207,124208,124209,124210,124211,124212,124213,124214,124215,124216,124217,124218,124219,124220,124221,124222,124223,124224,124225,124226,124227,124228,124229,124230,124231,124232,124233,124234,124235,expenditure
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [36]:
cosine_sim = cosine_similarity(x,x)

In [37]:
cosine_sim=pd.DataFrame(cosine_sim,index=custid,columns=custid)

In [38]:
cosine_sim.head()

,0,900,1800,1890,2700,2790,3600,3690,4500,4590,5400,5490,6300,6390,7200,7290,8100,8190,9090,9990,10890,11790,11880,12690,12780,13590,13680,14490,14580,15390,15480,16290,16380,17190,17280,18090,18180,19080,19179,19980,...,557145,557235,558045,558135,559035,559935,560835,561735,561825,562635,563535,563625,564435,564525,565335,565425,566235,566325,567135,567225,568035,568125,569025,569925,570825,571725,572625,573525,573615,574425,574515,575325,575415,576225,576315,577125,577215,578025,579015,580815
0,1.000000,0.851297,0.851757,0.632456,0.852104,0.800464,0.851404,0.849688,0.854415,0.802964,0.849972,0.850130,0.800261,0.854195,0.801327,0.850706,0.848528,0.851834,0.851132,0.854740,0.852323,0.855479,0.800000,0.851101,0.800869,0.851803,0.637865,0.801875,0.852994,0.801433,0.850013,0.849285,0.852263,0.850598,0.850810,0.632456,0.850434,0.850238,0.632456,0.851247,...,0.635228,0.800420,0.848528,0.801457,0.800632,0.850921,0.800000,0.801806,0.632456,0.633209,0.800395,0.632456,0.632456,0.850246,0.632456,0.632456,0.632456,0.632456,0.632456,0.632986,0.634405,0.636823,0.636485,0.633899,0.632456,0.640522,0.802856,0.634309,0.632456,0.632456,0.632456,0.634463,0.632456,0.632456,0.632456,0.632456,0.633443,0.632456,0.632456,0.632456
900,0.851297,1.000000,0.903086,0.670820,0.902903,0.852811,0.901541,0.901603,0.901094,0.852574,0.903555,0.902264,0.850551,0.904804,0.853954,0.904885,0.900687,0.902255,0.902797,0.903045,0.903850,0.904044,0.848528,0.903544,0.850327,0.902817,0.676518,0.849139,0.902379,0.853774,0.901250,0.901548,0.901926,0.903102,0.902716,0.670820,0.901910,0.904509,0.670820,0.903815,...,0.678601,0.849665,0.900000,0.849479,0.851165,0.903307,0.851247,0.850830,0.673293,0.678284,0.853269,0.670820,0.680508,0.901622,0.679195,0.671945,0.676489,0.671407,0.673373,0.676621,0.681779,0.672580,0.676537,0.674128,0.672384,0.679140,0.853069,0.678161,0.670820,0.670820,0.672905,0.675717,0.672512,0.672612,0.672891,0.671957,0.672403,0.672911,0.670820,0.670820
1800,0.851757,0.903086,1.000000,0.675108,0.903008,0.850498,0.901939,0.903317,0.901674,0.851521,0.903342,0.901720,0.850776,0.904306,0.852333,0.905637,0.901032,0.901340,0.903054,0.902936,0.903523,0.908517,0.848528,0.905010,0.851207,0.904194,0.673919,0.850312,0.900963,0.850745,0.901399,0.901896,0.902627,0.903935,0.902876,0.672419,0.902519,0.902858,0.670820,0.902574,...,0.674087,0.849814,0.900688,0.852030,0.849779,0.903315,0.851465,0.851726,0.674200,0.673845,0.851135,0.670820,0.673608,0.905996,0.670820,0.679436,0.674589,0.682820,0.672678,0.671696,0.674205,0.682161,0.678984,0.679042,0.672523,0.680623,0.855959,0.676253,0.670820,0.673878,0.670820,0.674426,0.672834,0.676168,0.671614,0.670820,0.674001,0.670820,0.670820,0.670820
1890,0.632456,0.670820,0.675108,1.000000,0.670820,0.636671,0.670820,0.670820,0.670820,0.632456,0.670820,0.670820,0.632456,0.670820,0.632456,0.670820,0.670820,0.670820,0.672017,0.670820,0.673055,0.670820,0.632456,0.670820,0.637792,0.670820,0.500000,0.632456,0.670820,0.632456,0.670820,0.675089,0.670820,0.670820,0.670820,0.500000,0.670820,0.670820,0.500000,0.670820,...,0.500000,0.632456,0.670820,0.632456,0.632456,0.670820,0.632456,0.632456,0.500000,0.500000,0.632456,0.500000,0.500000,0.670820,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.632456,0.500000,0.500000,0.500000,0.500000,0.500000,0.521808,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
2700,0.852104,0.902903,0.903008,0.670820,1.000000,0.850542,0.902334,0.900714,0.901801,0.851689,0.905457,0.903012,0.848834,0.904681,0.851198,0.904755,0.901505,0.901558,0.903515,0.903948,0.902763,0.904299,0.849262,0.904406,0.849989,0.902602,0.671959,0.850892,0.901463,0.855398,0.901894,0.900734,0.903839,0.914379,0.903513,0.670820,0.902358,0.905728,0.670820,0.904559,...,0.677725,0.852180,0.900000,0.850364,0.852124,0.901337,0.856428,0.851159,0.670820,0.679389,0.854677,0.670820,0.674786,0.901083,0.672302,0.670820,0.676869,0.674558,0.670820,0.677126,0.672677,0.674056,0.677702,0.679646,0.

### Now we have table containing a user-user similarity score



### Next we have to create a user-item matrix to find a users preference for all the products 

In [39]:
df_train.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,103057,103058,103059,103060,103061,103062,103063,103064,103065,103066,103067,103068,103069,103070,103071,103072,103073,103074,103075,103076,103077,103078,103079,103080,103081,103082,103083,103084,103085,103086,103087,103088,103089,103090,103091,103092,103093,103094,103095,103096
CustomerID,27270,27270,27270,27270,27270,27270,27270,427266,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,262737,262737,62739,37260,37260,37260,37260,37260,37260,37260,37260,37260,37260,...,376317,376317,376317,376317,183618,183618,183618,183618,183618,183618,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,306387,196398,3600,3600,3600,3600,3600,3600,3600,3600,3600,3600,3600,203598,203598,203598,203598
InvoiceNo,27270,27270,27270,27270,27270,27270,27270,427266,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,462735,262737,262737,62739,37260,37260,37260,37260,37260,37260,37260,37260,37260,37260,...,376317,376317,376317,376317,183618,183618,183618,183618,183618,183618,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,406386,306387,196398,3600,3600,3600,3600,3600,3600,3600,3600,3600,3600,3600,203598,203598,203598,203598
Quantity,7,7,9,7,7,2,7,3,28,28,14,14,28,57,28,21,28,28,28,28,24,28,28,14,28,28,43,7,7,38,7,7,9,7,7,7,7,7,2,4,...,9,9,1,3,28,2,4,4,3,28,7,1,1,4,2,12,1,1,43,57,28,1,28,57,48,14,14,19,3,9,4,28,30,14,14,4,14,9,14,4
InvoiceDate,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
UnitPrice,2.55,3.39,2.75,3.39,3.39,7.65,4.25,5.95,3.75,3.75,3.75,0.85,0.65,0.85,1.25,2.95,2.95,1.95,1.95,1.95,0.85,1.65,2.95,3.75,0.42,0.42,0.65,1.85,1.85,10.95,2.55,3.39,2.75,4.95,1.06,1.06,1.06,1.06,4.95,6.95,...,0.85,1.65,0.42,2.1,0.79,8.5,4.95,4.25,5.95,3.39,0.63,0.85,8.25,1.45,2.1,0.65,0.83,6.25,1.04,1.85,3.39,0.21,2.55,3.39,1.45,1.95,2.08,1.69,5.95,3.75,3.75,0.42,0.42,1.25,0.85,3.75,1.65,1.95,1.95,4.15
Country,PX,PX,PX,PX,PX,PX,PX,PX,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,RA,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,...,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,PX,RA,RA,RA,RA
StockCode,85123AY,71053R,84406BH,84029GV,84029EX,22752G,21730R,21756Q,22728B,22727B,22726C,21724T,21883V,10002F,21791H,21035O,22326L,22629C,22659F,22631M,22661W,21731V,22900U,21913C,22540W,22544C,22492E,22632O,22633V,21258H,85123AY,71053R,84406BH,20679N,37370Z,21871Y,21071P,21068Y,82483P,82486S,...,22573R,23274M,22153K,22141S,22464B,22627E,23355Z,21479J,23356F,21137F,23326S,21313C,48187N,23197D,21012K,22227K,23182N,21755A,23168M,21314Q,21137F,44234S,84347V,23328X,20725N,22178V,23084M,84879M,23356F,22726C,22727B,21916D,84692Z,84946F,21684X,23328X,22556N,22367K,22629C,23256O
expenditure,17.85,23.73,24.75,23.73,23.73,15.3,29.75,17.85,105,105,52.5,11.9,18.2,48.45,35,61.95,82.6,54.6,54.6,54.6,20.4,46.2,82.6,52.5,11.76,11.76,27.95,12.95,12.95,416.1,17.85,23.73,24.75,34.65,7.42,7.42,7.42,7.42,9.9,27.8,...,7.65,14.85,0.42,6.3,22.12,17,19.8,17,17.85,94.92,4.41,0.85,8.25,5.8,4.2,7.8,0.83,6.25,44.72,105.45,94.92,0.21,71.4,193.23,69.6,27.3,29.12,32.11,17.85,33.75,15,11.76,12.6,17.5,11.9,15,23.1,17.55,27.3,16.6
Custom_col1,PX12 85123AY12,PX12 71053R12,PX12 84406BH12,PX12 84029GV12,PX12 84029EX12,PX12 22752G12,PX12 21730R12,PX12 21756Q12,RA12 22728B12,RA12 22727B12,RA12 22726C12,RA12 21724T12,RA12 21883V12,RA12 10002F12,RA12 21791H12,RA12 21035O12,RA12 22326L12,RA12 22629C12,RA12 22659F12,RA12 22631M12,RA12 22661W12,RA12 21731V12,RA12 22900U12,RA12 21913C12,RA12 22540W12,RA12 22544C12,RA12 22492E

In [40]:
df_new=pd.pivot_table(df_train,index="CustomerID",columns="StockCode",values="Quantity")

In [41]:
df_new['Sum'] = df_new.sum(axis=1)

In [42]:
df_new.fillna(0,inplace=True)

In [43]:
df_new.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90205CD,90206AQ,90206CD,90208D,90209AX,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ,Sum
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2228.416667
900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5732.616667
1800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8722.450000
1890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.000000
2700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2605.400000


In [44]:
df_new=df_new.div(df_new.Sum, axis=0)

In [45]:
df_new.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90205CD,90206AQ,90206CD,90208D,90209AX,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ,Sum
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000449,0.000449,0.0,0.0,0.0,0.0,0.0,0.0,0.000449,0.000449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010466,0.000174,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000523,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000229,0.0,0.001605,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000384,0.000000,0.0,0.005373,0.0,0.0,0.0,0.0,0.0,0.000000,0.006141,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [46]:
df_new.drop(["Sum"],axis=1,inplace=True)

In [47]:
table=df_new.T
index1=df_new.columns
col1=df_new.index
table = scaler.fit_transform(table)
table=pd.DataFrame(table,index=index1,columns=col1)
table.head()

CustomerID,0,900,1800,1890,2700,2790,3600,3690,4500,4590,5400,5490,6300,6390,7200,7290,8100,8190,9090,9990,10890,11790,11880,12690,12780,13590,13680,14490,14580,15390,15480,16290,16380,17190,17280,18090,18180,19080,19179,19980,...,557145,557235,558045,558135,559035,559935,560835,561735,561825,562635,563535,563625,564435,564525,565335,565425,566235,566325,567135,567225,568035,568125,569025,569925,570825,571725,572625,573525,573615,574425,574515,575325,575415,576225,576315,577125,577215,578025,579015,580815
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002F,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10080Q,0.0,0.0,0.0,0.0,0.0,0.0,0.038889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10120M,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002778,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10123CU,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10124AW,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
df_new=table.T
df_new.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90205AR,90205CD,90206AQ,90206CD,90208D,90209AX,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.008696,0.008696,0.0,0.0,0.0,0.0,0.0,0.0,0.008696,0.008696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.004167,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.012500,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.002315,0.0,0.016204,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.008333,0.000000,0.0,0.116667,0.0,0.0,0.0,0.0,0.0,0.000000,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
test_customers=pd.DataFrame(df_test.CustomerID.unique(),columns=["CustomerID"])
test_customers.head()

,CustomerID
0,127269
1,227268
2,327267
3,162738
4,252747


In [50]:
df=df_train[["CustomerID","StockCode"]]
df.StockCode=df.StockCode+" "
d = {'StockCode': 'sum'}
df = df.groupby('CustomerID', as_index=False).aggregate(d).reindex(columns=df.columns)
df.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,CustomerID,StockCode
0,0,21429E 21428B 22961C 84375E 21877C 21870X 2186...
1,900,22943F 22118J 22121N 22119D 21485W 22219N 2152...
2,1800,22655R 84985AB 22113T 21916D 22219N 22294U 211...
3,1890,21094A 21080R 48194E 21770E 22851J 22908G 2241...
4,2700,22697A 21877C 21071P 22072Y 21527P 22151L 2248...


In [51]:
df['StockCode'] = df['StockCode'].str.strip('()').str.split(' ')
df.head()

,CustomerID,StockCode
0,0,"[21429E, 21428B, 22961C, 84375E, 21877C, 21870..."
1,900,"[22943F, 22118J, 22121N, 22119D, 21485W, 22219..."
2,1800,"[22655R, 84985AB, 22113T, 21916D, 22219N, 2229..."
3,1890,"[21094A, 21080R, 48194E, 21770E, 22851J, 22908..."
4,2700,"[22697A, 21877C, 21071P, 22072Y, 21527P, 22151..."


In [52]:
##generate similar user list for a given user
def sim_users(UserID):
    #id=UserID.astype('int')
    ###rank all the stocks in descending order 
    users=list(cosine_sim.loc[UserID].sort_values(ascending=False).index)
    
    ###remove all the  stocks from stock_ranking list that have already been purchased by the user 
    recommendations = [x for x in users if x!=UserID]
    recommendations=recommendations[:15]
    return recommendations

In [53]:
##generate the recommended products based on the purchase histories of the similar users
def sim_prod(user_list,original_pur,userid):
    weight=1
    ##assigning different weights to the user preferences based on their rankings in the list
    
    resultdf=pd.DataFrame(columns=df_new.columns)
    prod1=list(preds.loc[userid])
    m = np.asarray(prod1)
    
    for i in user_list:
        resultdf=resultdf.append(df_new[df_new.index==i])
    
    resultdf=resultdf.T
   
    for i in user_list:
        resultdf[i]=resultdf[i]*weight
        weight-=0.06
    ## Adding all the scores given by the recommended users to all the products 
    ## and selecting the products with the best scores 
   
    resultdf["Sum"]=resultdf.sum(axis = 1, skipna = True)
    resultdf["Sum1"]=m
    resultdf["Sum"]=resultdf.Sum+resultdf.Sum1
        
    prod_list=list(resultdf["Sum"].sort_values(ascending=False).index)
    recommendations = [x for x in prod_list if x not in original_pur]
    recommendations=recommendations[:400]
    return recommendations

In [54]:
df=pd.merge(df, test_customers, how='inner', on=["CustomerID"])

In [55]:
df.head()

,CustomerID,StockCode
0,1890,"[21094A, 21080R, 48194E, 21770E, 22851J, 22908..."
1,2700,"[22697A, 21877C, 21071P, 22072Y, 21527P, 22151..."
2,3600,"[22745M, 22747N, 22699D, 47480X, 20749K, 21935..."
3,3690,"[90175CP, 90210BN, 90175AH, 90002DN, 22297H, 7..."
4,4500,"[22622G, 22536B, 21791H, 22659F, 21731V, 22326..."


In [56]:
df["Items"]=None
df.head()

,CustomerID,StockCode,Items
0,1890,"[21094A, 21080R, 48194E, 21770E, 22851J, 22908...",None
1,2700,"[22697A, 21877C, 21071P, 22072Y, 21527P, 22151...",None
2,3600,"[22745M, 22747N, 22699D, 47480X, 20749K, 21935...",None
3,3690,"[90175CP, 90210BN, 90175AH, 90002DN, 22297H, 7...",None
4,4500,"[22622G, 22536B, 21791H, 22659F, 21731V, 22326...",None


In [57]:
##generating recommendation list for all test users
for i in df.index:
    userid=df.loc[i,"CustomerID"]
    stock=df.loc[i,"StockCode"]
    user_list=sim_users(userid)
    prod_list=sim_prod(user_list,stock,userid)
    df.set_value(i, 'Items', prod_list)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

...

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  i

In [58]:
df

,CustomerID,StockCode,Items
0,1890,"[21094A, 21080R, 48194E, 21770E, 22851J, 22908...","[17003M, 23202Z, 21985C, 20713F, 22616B, 84077..."
1,2700,"[22697A, 21877C, 21071P, 22072Y, 21527P, 22151...","[22178V, 23310X, 22491V, 22264D, 22741O, 23308..."
2,3600,"[22745M, 22747N, 22699D, 47480X, 20749K, 21935...","[21212D, 21829T, 23077P, 21162S, 22469S, 22923..."
3,3690,"[90175CP, 90210BN, 90175AH, 90002DN, 22297H, 7...","[17003M, 84978X, 22610S, 22560W, 22991T, 21650..."
4,4500,"[22622G, 22536B, 21791H, 22659F, 21731V, 22326...","[84077K, 20713F, 22492E, 23035A, 22469S, 23026..."
5,5490,"[22469S, 22633V, 22952P, 22800V, 22801J, 21326...","[22151L, 84077K, 22610S, 22554P, 21124F, 22386..."
6,8190,"[22346F, 21787C, 22629C, 22837N, 85099BJ, 2065...","[22469S, 21977R, 22030B, 21790Y, 22727B, 22728..."
7,9090,"[15056pC, 18098cC, 20615X, 20665P, 20679N, 209...","[22086O, 23084M, 22577B, 21135E, 17096D, 22616..."
8,13680,"[15056BLH, 22294U, 21889K, 85123AY, 84879M, 84...","[22578A, 22577B, 22355U, 22579W, 71459J, 20723..."
9,14490,"[84795BQ, 79191BB, 22585G, 20771Z, 20765L, 207...","[84077K, 84755J, 84347V, 21243M, 23084M, 84945..."


In [59]:
df.drop(columns=["StockCode"],axis=1,inplace=True)

In [60]:
df.head()

,CustomerID,Items
0,1890,"[17003M, 23202Z, 21985C, 20713F, 22616B, 84077..."
1,2700,"[22178V, 23310X, 22491V, 22264D, 22741O, 23308..."
2,3600,"[21212D, 21829T, 23077P, 21162S, 22469S, 22923..."
3,3690,"[17003M, 84978X, 22610S, 22560W, 22991T, 21650..."
4,4500,"[84077K, 20713F, 22492E, 23035A, 22469S, 23026..."


In [61]:
df.to_csv('submission.csv', index=False)